In [1]:
from bs4 import BeautifulSoup

dropdown_html = '''
<select id="ctl00_ContentPlaceHolder1_Result1_ddlState" name="state" onchange="return GetResult(this)">
<option value=""> Select State Wise </option>
<option value="U01">Andaman &amp; Nicobar Islands</option>
<option value="S01">Andhra Pradesh</option>
<option value="S02">Arunachal Pradesh</option>
<option value="S03">Assam</option>
<option value="S04">Bihar</option>
<option value="U02">Chandigarh</option>
<option value="S26">Chhattisgarh</option>
<option value="U03">Dadra &amp; Nagar Haveli and Daman &amp; Diu</option>
<option value="S05">Goa</option>
<option value="S06">Gujarat</option>
<option value="S07">Haryana</option>
<option value="S08">Himachal Pradesh</option>
<option value="U08">Jammu and Kashmir</option>
<option value="S27">Jharkhand</option>
<option value="S10">Karnataka</option>
<option value="S11">Kerala</option>
<option value="U09">Ladakh</option>
<option value="U06">Lakshadweep</option>
<option value="S12">Madhya Pradesh</option>
<option value="S13">Maharashtra</option>
<option value="S14">Manipur</option>
<option value="S15">Meghalaya</option>
<option value="S16">Mizoram</option>
<option value="S17">Nagaland</option>
<option value="U05">NCT OF Delhi</option>
<option value="S18">Odisha</option>
<option value="U07">Puducherry</option>
<option value="S19">Punjab</option>
<option value="S20">Rajasthan</option>
<option value="S21">Sikkim</option>
<option value="S22">Tamil Nadu</option>
<option value="S29">Telangana</option>
<option value="S23">Tripura</option>
<option value="S24">Uttar Pradesh</option>
<option value="S28">Uttarakhand</option>
<option value="S25">West Bengal</option>
</select>
'''

soup = BeautifulSoup(dropdown_html, 'html.parser')
dropdown = soup.find('select', id='ctl00_ContentPlaceHolder1_Result1_ddlState')

state_links = {}
if dropdown:
    options = dropdown.find_all('option')
    for option in options:
        option_value = option.get('value')
        if option_value:  # Check if option_value is not None or empty
            state_name = option.text.strip()
            state_links[state_name] = 'https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-' + option_value + '.htm'
else:
    print("Dropdown not found.")

# Print the dictionary to see the result
for state, link in state_links.items():
    print(f'{state}: {link}')


Andaman & Nicobar Islands: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U01.htm
Andhra Pradesh: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S01.htm
Arunachal Pradesh: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S02.htm
Assam: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S03.htm
Bihar: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S04.htm
Chandigarh: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U02.htm
Chhattisgarh: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S26.htm
Dadra & Nagar Haveli and Daman & Diu: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U03.htm
Goa: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S05.htm
Gujarat: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S06.htm
Haryana: https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S07.htm
Himachal Pradesh: https://results.eci.gov.in/PcResultGen

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import os

def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        page_content = response.content
        soup = BeautifulSoup(page_content, 'html.parser')
        return soup
    else:
        print(f"Failed to retrieve {url}")
        return None

def extract_data_to_csv(soup, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        tables = soup.find_all('table')
        for table in tables:
            if table.find('th'): 
                headers = [th.text.strip() for th in table.find_all('th')]
                writer.writerow(headers)

                rows = table.find_all('tr')[1:]  
                for row in rows:
                    columns = row.find_all('td')
                    row_data = [column.text.strip() for column in columns]
                    writer.writerow(row_data)

output_folder = 'csv_statefiless'
os.makedirs(output_folder, exist_ok=True)

for state_name, url in state_links.items():
    print(f"Fetching data from {url}")
    soup = fetch_data(url)
    if soup:
        # Replace spaces with underscores and remove any special characters for the filename
        safe_state_name = state_name.replace(" ", "_").replace("&", "and")
        filename = os.path.join(output_folder, f"{safe_state_name}.csv")
        extract_data_to_csv(soup, filename)
        print(f"Data written to {filename}")
